In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
MIMIC-IV Sepsis Cohort Preprocessing (for uncompressed .csv files)

Description:
This script executes Phase 1 of the data processing pipeline. It builds the
Sepsis cohort from the MIMIC-IV dataset, extracts relevant features from the
first 24 hours of each patient's ICU stay, and generates a final feature matrix.

Output:
- 'sepsis_feature_matrix.csv': A CSV file where each row corresponds to a unique
  ICU stay, containing static features, aggregated vitals, and lab values.
  Missing values are represented as NaN, ready for imputation in the next phase.
"""
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import timedelta

# --- Constants & Configuration ---

# Paths to the MIMIC-IV data directories ON GOOGLE DRIVE.
# !!! ENSURE THESE PATHS ARE CORRECT FOR YOUR DRIVE SETUP !!!
DRIVE_BASE_PATH = "D:\mimic-iv-3.1\mimic-iv-3.1"
HOSP_DATA_PATH = Path(DRIVE_BASE_PATH + "\csv files\hosp")
ICU_DATA_PATH = Path(DRIVE_BASE_PATH + "\csv files\icu")

# Define where the output file will be saved in your Drive.
OUTPUT_PATH = Path("../data/preprocessed/")




# Time window for feature extraction from ICU admission.
TIME_WINDOW_HOURS = 24

# Clinical variables defined from the project's data analysis phase.
SEPSIS_ICD_CODES = [
    'A021', 'A227', 'A267', 'A327', 'A400', 'A401', 'A403', 'A408', 'A409',
    'A4101', 'A4102', 'A411', 'A412', 'A413', 'A414', 'A4150', 'A4151',
    'A4152', 'A4153', 'A4159', 'A4181', 'A4189', 'A419', 'A427', 'A5486',
    'B377', 'O0337', 'O0387', 'O0487', 'O0737', 'O0882', 'O85', 'O8604',
    'P360', 'P3610', 'P3619', 'P362', 'P3630', 'P3639', 'P364', 'P365',
    'P368', 'P369', 'R6520', 'R6521', 'T8144', 'T8144XA', 'T8144XD', 'T8144XS'
]

LAB_PANEL_ITEMIDS = {
    'cbc_hematocrit': 51221, 'cbc_hemoglobin': 51222, 'cbc_platelet': 51265,
    'cbc_rbc': 51279, 'cbc_wbc': 51301, 'cmp_bicarbonate': 50882,
    'cmp_creatinine': 50912, 'cmp_glucose': 50931, 'cmp_potassium': 50971,
    'cmp_bun': 51006, 'cmp_aniongap': 50868, 'cmp_lactate': 50813,
    'abg_ph': 50820, 'abg_o2_saturation': 50817, 'abg_base_excess': 50802,
    'aptt_ptt': 51275, 'aptt_inr': 51237
}

VITAL_SIGN_ITEMIDS = {
    'heart_rate': 220045, 'sbp': 220179, 'dbp': 220180,
    'respiratory_rate': 220210, 'temperature_c': 223761, 'spo2': 220277
}

# Invert dictionaries for easy name lookup
ITEMID_TO_LAB_NAME = {v: k for k, v in LAB_PANEL_ITEMIDS.items()}
ITEMID_TO_VITAL_NAME = {v: k for k, v in VITAL_SIGN_ITEMIDS.items()}


def load_sepsis_cohort(hosp_path):
    """
    Identifies the Sepsis cohort using ICD-10 codes and merges demographic
    and admission data to create a base cohort table.
    """
    print("Loading cohort...")
    diagnoses = pd.read_csv(hosp_path / 'diagnoses_icd.csv')
    sepsis_subjects = diagnoses[diagnoses['icd_code'].isin(SEPSIS_ICD_CODES)]['subject_id'].unique()

    patients = pd.read_csv(hosp_path / 'patients.csv', usecols=['subject_id', 'gender', 'anchor_age', 'anchor_year'])
    admissions = pd.read_csv(hosp_path / 'admissions.csv', usecols=['subject_id', 'hadm_id', 'admittime', 'hospital_expire_flag'])
    admissions['admittime'] = pd.to_datetime(admissions['admittime'])

    # Filter for sepsis patients and calculate age at admission
    sepsis_admissions = admissions[admissions['subject_id'].isin(sepsis_subjects)].copy()
    sepsis_admissions = pd.merge(sepsis_admissions, patients, on='subject_id')
    sepsis_admissions['age'] = sepsis_admissions['anchor_age'] + (sepsis_admissions['admittime'].dt.year - sepsis_admissions['anchor_year'])

    print(f"Identified {len(sepsis_subjects)} unique patients and {len(sepsis_admissions)} sepsis-related admissions.")
    return sepsis_admissions


def link_cohort_to_icu_stays(cohort_df, icu_path):
    """
    Links hospital admissions to ICU stays and defines the 24-hour feature
    extraction window for each stay.
    """
    print("Linking cohort to ICU stays...")
    icustays = pd.read_csv(icu_path / 'icustays.csv', usecols=['hadm_id', 'stay_id', 'intime'])
    icustays['intime'] = pd.to_datetime(icustays['intime'])

    cohort_icu = pd.merge(cohort_df, icustays, on='hadm_id')
    cohort_icu['endtime'] = cohort_icu['intime'] + timedelta(hours=TIME_WINDOW_HOURS)

    return cohort_icu.drop_duplicates(subset=['hadm_id', 'stay_id'])


def extract_vitals_for_stays(stay_windows_df, icu_path):
    """
    Extracts and aggregates vital signs from chartevents for the given ICU stays.
    """
    print("Extracting vitals from chartevents (this may take a few minutes)...")
    vitals_all = []
    # Process the large chartevents file in chunks
    for chunk in pd.read_csv(icu_path / 'chartevents.csv',
                             usecols=['stay_id', 'itemid', 'charttime', 'valuenum'],
                             chunksize=10_000_000, low_memory=False):

        chunk.dropna(subset=['valuenum'], inplace=True)
        chunk = chunk[chunk['stay_id'].isin(stay_windows_df['stay_id'])]
        chunk = chunk[chunk['itemid'].isin(VITAL_SIGN_ITEMIDS.values())]

        if not chunk.empty:
            chunk['charttime'] = pd.to_datetime(chunk['charttime'])
            chunk_merged = pd.merge(chunk, stay_windows_df, on='stay_id', how='left')
            vitals_in_window = chunk_merged[
                (chunk_merged['charttime'] >= chunk_merged['intime']) &
                (chunk_merged['charttime'] <= chunk_merged['endtime'])
            ]
            vitals_all.append(vitals_in_window[['stay_id', 'itemid', 'valuenum']])

    vitals_df = pd.concat(vitals_all)

    # Aggregate vitals to get mean, min, and max over the window
    vitals_agg = vitals_df.groupby(['stay_id', 'itemid'])['valuenum'].agg(['mean', 'min', 'max']).unstack()
    vitals_agg.columns = [f"{ITEMID_TO_VITAL_NAME[itemid]}_{stat}" for stat, itemid in vitals_agg.columns]

    return vitals_agg


def extract_labs_for_stays(stay_windows_df, hosp_path):
    """
    Extracts and aggregates lab measurements from labevents for the given ICU stays.
    """
    print("Extracting labs from labevents (this may take a few minutes)...")
    relevant_hadm_ids = stay_windows_df['hadm_id'].unique()
    labs_all = []

    # Process the large labevents file in chunks
    for chunk in pd.read_csv(hosp_path / 'labevents.csv',
                             usecols=['hadm_id', 'itemid', 'charttime', 'valuenum'],
                             chunksize=10_000_000, low_memory=False):

        chunk.dropna(subset=['valuenum'], inplace=True)
        chunk = chunk[chunk['hadm_id'].isin(relevant_hadm_ids)]
        chunk = chunk[chunk['itemid'].isin(LAB_PANEL_ITEMIDS.values())]

        if not chunk.empty:
            chunk['charttime'] = pd.to_datetime(chunk['charttime'])
            chunk_merged = pd.merge(chunk, stay_windows_df, on='hadm_id', how='left')
            labs_in_window = chunk_merged[
                (chunk_merged['charttime'] >= chunk_merged['intime']) &
                (chunk_merged['charttime'] <= chunk_merged['endtime'])
            ]
            labs_all.append(labs_in_window[['stay_id', 'itemid', 'valuenum']])

    labs_df = pd.concat(labs_all)

    # Aggregate labs to get the mean value over the window
    labs_agg = labs_df.groupby(['stay_id', 'itemid'])['valuenum'].mean().unstack()
    labs_agg.columns = [ITEMID_TO_LAB_NAME[itemid] for itemid in labs_agg.columns]

    return labs_agg


def main():
    """Main execution pipeline."""
    OUTPUT_PATH.mkdir(exist_ok=True, parents=True)

    # 1. Identify cohort and get static data
    sepsis_admissions = load_sepsis_cohort(HOSP_DATA_PATH)

    # 2. Link to ICU stays to define time windows
    sepsis_icu = link_cohort_to_icu_stays(sepsis_admissions, ICU_DATA_PATH)
    stay_windows = sepsis_icu[['stay_id', 'hadm_id', 'intime', 'endtime']].copy()

    # 3. Extract and aggregate time-series features
    vitals_features = extract_vitals_for_stays(stay_windows, ICU_DATA_PATH)
    labs_features = extract_labs_for_stays(stay_windows, HOSP_DATA_PATH)

    # 4. Assemble the final feature matrix
    print("Assembling final feature matrix...")
    base_data = sepsis_icu.set_index('stay_id')
    final_matrix = base_data.join(vitals_features).join(labs_features)

    # 5. Clean up and select final columns
    final_matrix['gender'] = final_matrix['gender'].apply(lambda x: 1 if x == 'M' else 0)

    static_cols = ['subject_id', 'hadm_id', 'age', 'gender', 'hospital_expire_flag']
    feature_cols = list(vitals_features.columns) + list(labs_features.columns)
    # Ensure all expected feature columns are present, filling with NaN if not
    for col in feature_cols:
        if col not in final_matrix:
            final_matrix[col] = np.nan

    final_matrix = final_matrix[static_cols + feature_cols]

    # 6. Save the output
    output_file = OUTPUT_PATH / 'sepsis_feature_matrix.csv'
    final_matrix.to_csv(output_file)

    print("\n--- Preprocessing Complete ---")
    print(f"Final matrix shape: {final_matrix.shape}")
    print(f"Saved to: {output_file}")


# Run the main function
main()

Loading cohort...
Identified 11321 unique patients and 61530 sepsis-related admissions.
Linking cohort to ICU stays...
Extracting vitals from chartevents (this may take a few minutes)...
Extracting labs from labevents (this may take a few minutes)...
Assembling final feature matrix...

--- Preprocessing Complete ---
Final matrix shape: (16780, 40)
Saved to: data\preprocessed\sepsis_feature_matrix.csv
